In [1]:
%pip install crewai crewai_tools langchain_community python-dotenv
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from langchain_groq import ChatGroq
from utils import get_groq_api_key
from crewai_tools import ScrapeWebsiteTool
from IPython.display import Markdown
from crewai import Agent, Task, Crew

groq_api_key = get_groq_api_key()


llm = ChatGroq(
    model="groq/llama3-8b-8192",
    temperature=0
)

In [4]:
docs_scrape_tool = ScrapeWebsiteTool()

#### Agents
* Planner agent
* Researcher agent
* Writer agent
* Proofreader agent

In [5]:
planner = Agent(
    llm=llm,
    role="Resume Strategy Coordinator",
    goal=f"""
    Define structure and roadmap

    Create a roadmap detailing sections to include (e.g., contact details, objective, skills, education, experience, projects, achievements)

    Identify any gaps that require additional input or materials
    """,
    backstory="As a Resume Strategy Coordinator, you are responsible for planning the structure of the resume based on the purpose of the resume and the target audience.",
    max_iter=2,
    verbose=True
)

In [6]:
researcher = Agent(
    llm=llm,
    role="Resume Market Researcher",
    goal=f"""
    Gather Information and Trends
    
    Collect industry-specific keywords, trends and formatting styles.
    Analyse top-tier resume in the desired field for inspiration.
    Gather detailed information about client's accomplistments, skills and projects.
    """,
    backstory="As a Resume Market Researcher, you are responsible for researching the best resumes for the industry and take inspiration from them.",
    max_iter=2,
    verbose=True
)

In [7]:
writer = Agent(
    llm=llm,
    role="Resume Content Specialist",
    goal=f"""
    Draft the resume

    Craft a compelling draft based on the Resume Market Researcher's input and Resume Strategy Coordinator's structure.
    Tailor language to highlight strengths while aligning with the researched industry trends.
    """,
    backstory="As a Resume Content Specialist, you are responsible to craft a compelling, tailored resume that highlights client's skills, achievements, and potential, aligning with the technology industry's expectations.",
    verbose=True,
    max_iter=2
)

In [8]:
proofreader = Agent(
    llm=llm,
    role="Resume Quality Assurance Specialist",
    goal=f"""
    Refine and validate

    Check for grammer, clarity, tone and consistency.
    Ensure that the key points stand out and there's no uncessary repetition.
    Validate alignment with the target audience's expectations.
    """,
    backstory="As a Resume Quality Assurance Specialist, ensure the resume is error-free, professional, and polished, with consistent tone and formatting throughout.",
    verbose=True,
    max_iter=2
)

#### Tasks
* Planning task
* Researching task
* Writing task
* Proofreading task

In [9]:
planning = Task(
    description="""
    Collaborate with client to know the purpose and target audience of the resume.
    - Purpose: {purpose}
    - Target Audience: {target_audience}

    Outline the structure of the resume, including sections and their sequence 
    Develop a plan for updating the resume based on future feedback or new achievements.
    """,
    agent=planner,
    expected_output="A structured outline specifying the order and sections to include."
)

In [10]:
researching = Task(
    description="""
    Research job descriptions and identify common keywords for the desired roles.
    Study examples of successful resumes in the {industry} industry.
    Gather and organize client's professional achievements, metrics, and details for use in the resume.

    -Work Experience: {work_experience}
    -Skills: {skills}
    -Projects worked on: {projects}
    -Education: {educations}
    -Social media links: {social_media_links}
    -Certification: {certifications}
    """,
    agent=researcher,
    expected_output="A document or presentation outlining current trends in resume formatting, especially those relevant to the tech industry.",
    tools=[docs_scrape_tool]
)

In [11]:
writing = Task(
    description="""
    Write concise,impactful content for each section of the resume based on the Resume Market Researcher's input and Resume Strategy Coordinator's structure.
    Tailor the language and tone to fit the desired job roles in the technology industry.
    Ensure keywords and phrases match industry standards and ATS requirements.
    Job roles: {job_roles}
    """,
    agent=writer,
    expected_output="Versions of the resume tailored for different {job_roles}, focusing on the most relevant skills and experience."
)

In [12]:
proofreading = Task(
    description="""
    Check for grammer, punctuation and spelling errors.
    Verify the clarity, readability and conciseness of the content.
    Cross-check information for accuracy (e.g., dates, job titles, metrics).
    Ensure that the resume adheres to professional standards and conveys confidence.
    """,
    agent=proofreader,
    expected_output="A final document where all dates, job titles, and achievements are fact-checked and correctly represented."
)

In [13]:
inputs = {
    "purpose":"To secure a challenging position in the technology industry, focusing on Software Engineering, where I can apply my skills and knowledge to contribute to innovative projects and grow as a professional.",
    "target_audience":"Hiring Managers",
    "industry":"IT industry",
    "work_experience": """
                        I worked as a mentee in the CNCF program I worked under project called Konveyor AI: Data Querying for Kai & InstructLab Integration Potential.I worked on to solve issues with the LLM's response.
                        """,
    "job_roles":"Software Engineer",
    "skills": ["Podman","Git","Linux Fedora","Python","LLM agents","Prompt Engineering"],
    "projects":["I worked on a pizza AI chatbot with speech and voice recognition"],
    "educations":["I have completed O-Levels as one of the Top achiever in the country","I am a recent high school gradute with Top achivers in the county"],
    "social_media_links":["https://github.com/devjpt23"],
    "certifications":"I have completed a CS50 Python Course by Harvard."
}

In [14]:
mygang = Crew(
    agents=[planner, proofreader, writer, researcher],
    tasks=[planning, proofreading, writing, researching],
    max_rpm=29
)

In [17]:
results = mygang.kickoff(inputs=inputs)

# Agent: Resume Strategy Coordinator
## Task: 
    Collaborate with client to know the purpose and target audience of the resume.
    - Purpose: To secure a challenging position in the technology industry, focusing on Software Engineering, where I can apply my skills and knowledge to contribute to innovative projects and grow as a professional.
    - Target Audience: Hiring Managers

    Outline the structure of the resume, including sections and their sequence 
    Develop a plan for updating the resume based on future feedback or new achievements.
    


# Agent: Resume Strategy Coordinator
## Final Answer: 
**Resume Structure:**

1. **Contact Information:**
	* Full name
	* Professional email address
	* Phone number
	* LinkedIn profile (or other relevant social media)
	* Address (optional)
2. **Summary/Objective Statement:**
	* A brief overview of the candidate's experience, skills, and career goals (approx. 2-3 sentences)
	* Tailored to the target audience (Hiring Managers) and purp

In [18]:
Markdown(results.raw)

**Current Trends in Resume Formatting for the Tech Industry:**

1. **Summary/Objective Statement:** A concise overview of the candidate's experience, skills, and achievements. It should be tailored to the specific job being applied for and highlight the candidate's unique value proposition.

2. **Technical Skills:** A comprehensive list of programming languages, development frameworks, databases, operating systems, and other technical skills relevant to the job. It's essential to prioritize the most relevant skills and avoid overwhelming the reader with too many.

3. **Work Experience:** A reverse chronological list of work experience, with the most recent experience first. Each entry should include a brief description of the job, the company, and the candidate's achievements and responsibilities.

4. **Education:** A list of relevant educational background, including degrees, institutions, and relevant coursework.

5. **Projects:** A showcase of personal or professional projects that demonstrate the candidate's skills and achievements. It's essential to include a brief description of the project, the technologies used, and the outcome.

6. **Achievements:** A list of notable achievements, such as awards, certifications, or publications. It's essential to highlight the most relevant and impressive achievements.

7. **References:** A list of professional references, including names, titles, companies, and contact information. It's essential to obtain permission from the references before listing them.

8. **Plan for Updating the Resume:** A plan for regularly updating the resume, including reviewing and revising it every 6-12 months, and updating it to reflect changes in the job market, industry trends, or new technologies.

**Additional Tips:**

* Use a clear and concise writing style, avoiding jargon and technical terms that may be unfamiliar to non-technical readers.
* Use bullet points and short paragraphs to make the resume easy to scan and read.
* Prioritize the most relevant information and avoid overwhelming the reader with too much detail.
* Use action verbs such as "Designed," "Developed," "Implemented," and "Managed" to describe achievements and responsibilities.
* Include relevant hobbies or interests that demonstrate transferable skills or personality traits.

By following these trends and tips, you can create a resume that effectively showcases your skills, achievements, and experience, and helps you stand out in the competitive tech job market.